In [4]:
from pathlib import Path
from glob import glob
import pickle
import numpy as np
import os

In [5]:
path = f"{Path(os.getcwd()).parents[0]}/data/*.npy"
for path_name in glob(path):
    print(path_name)

/home/arnav/Disk/abhishek/Code/EEG_BiLSTM/data/Channel_lateralization_map.npy
/home/arnav/Disk/abhishek/Code/EEG_BiLSTM/data/data(5-95).npy
/home/arnav/Disk/abhishek/Code/EEG_BiLSTM/data/data(5-95)_old.npy
/home/arnav/Disk/abhishek/Code/EEG_BiLSTM/data/label.npy


In [9]:
data_path = glob(path)[1]
label_path = glob(path)[3]

data = np.load(data_path)
label = np.load(label_path)

In [12]:
import pandas as pd
label_df = pd.DataFrame(label, columns = ["cls"])

In [37]:
a = list(label_df["cls"].value_counts().index)
a.sort()

In [ ]:
a

In [4]:
lat_dict_path = Path(os.getcwd()).parents[0]/"data/lat_dict.pkl"
print(lat_dict_path)
with open(lat_dict_path, "rb") as fp:
   lat_dict = pickle.load(fp)

/home/arnav/Disk/abhishek/Code/EEG_BiLSTM/data/lat_dict.pkl


## Parameters

In [5]:
from utils import regional_info_extraction

In [6]:
feature_extraction_shape = len(lat_dict["left_hms"])+len(lat_dict["middle_hms"])
print(f'Feature Extraction Module Output Shape(No. Feature): {feature_extraction_shape}')

Feature Extraction Module Output Shape(No. Feature): 85


In [7]:
regional_info_transform = regional_info_extraction(**lat_dict)
data_ = regional_info_transform.fit_transform(data)

In [8]:
num_samples =data_.shape[0]
num_samples

11964

In [9]:
train_frac = 0.90
test_frac = 0.10

In [10]:
train_size = round(num_samples*train_frac)
test_size = num_samples - train_size
print(f"Train size: {train_size}")
print(f"Test size: {test_size}")

Train size: 8375
Test size: 3589


## Train/Test Split

In [63]:
# Train
X_train = data_[:train_size]
y_train = label[:train_size]
print(f"Train Size: X {X_train.shape} || y {y_train.shape}")


#Test
X_test = data_[train_size :]
y_test = label[train_size :]
print(f"Test Size: X {X_test.shape} || y {y_test.shape}")

Train Size: X (8375, 440, 85) || y (8375,)
Test Size: X (3589, 440, 85) || y (3589,)


In [64]:
pro_data_label_dict = {
    "train": {
        "X": X_train,
        "y": y_train
    },
    "test" : {
        "X" : X_test,
        "y" : y_test
    }
}

In [70]:
with open(f"{Path(os.getcwd()).parents[0]}/data/train_test_data.pkl", "wb") as f:
           pickle.dump(pro_data_label_dict, f)